---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas - Test
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [281]:
import pandas as pd
import numpy as np

def merge_and_clean_datasets():
    energy_df = pd.read_excel("Energy Indicators.xls", skiprows=17, skip_footer=38)
    bank_df = pd.read_csv("world_bank.csv", skiprows=4)
    sci_df = pd.read_excel("scimagojr-3.xlsx")

    energy_df = energy_df.drop("Unnamed: 0", axis=1)\
                         .drop("Unnamed: 1", axis=1)

    energy_df.columns = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]

    countries_to_rename = d = {
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    }

    # Convert to gigajoules
    energy_df["Energy Supply"] *= 1000000

    # Remove all numbers
    energy_df["Country"] = energy_df["Country"].str.replace('\d+', '')
    # Remove all brackets and spaces after
    energy_df["Country"] = energy_df["Country"].str.replace('\([^)]*\)', '')
    energy_df["Country"] = energy_df["Country"].str.rstrip()


    # Rename all the long country names
    energy_df["Country"] = energy_df["Country"].apply(lambda x: d[x] if x in d else x)

    countries_to_rename = f = {
        "Korea, Rep.": "South Korea", 
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"
    }

    bank_df["Country Name"] = bank_df["Country Name"].apply(lambda x: f[x] if x in f else x)
    bank_df.rename(columns={ "Country Name": "Country" }, inplace=True)

    sci_df.rename(columns={ "country": "Country" }, inplace=True)

    df = pd.merge(pd.merge(energy_df,bank_df,on="Country"),sci_df,on="Country")
    df.set_index(["Country"], inplace=True)

    for year in range(1960, 2006):
        df.drop(str(year), axis=1, inplace=True)

    df = df[["Rank", "Documents", "Citable documents", "Citations","Self-citations",
     "Citations per document", "H index","Energy Supply","Energy Supply per Capita","% Renewable",
     "2006","2007","2008","2009", "2010", "2011", "2012", "2013", "2014", "2015"]]
    return energy_df, bank_df, sci_df, df

def answer_one():
    e, b, s, df = merge_and_clean_datasets()
    df = df[df["Rank"] <= 15].sort_values(by=["Rank"])

    return df

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [287]:
def answer_two():
    e, b, s, df = merge_and_clean_datasets()
    b.rename(columns={ "Country Name": "Country" }, inplace=True)
    s.rename(columns={ "country": "Country" }, inplace=True)

    union = pd.merge(pd.merge(e, b, on='Country', how='outer'), s, on='Country', how='outer')

    return len(union) - len(df)

<br>

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [233]:
def answer_three():
    keys = [str(x) for x in range(2006, 2016)]
    Top15 = answer_one()
    Top15['avgGDP'] = Top15[keys].mean(axis=1)
    
    return Top15['avgGDP'].sort_values(ascending=False)

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [290]:
def answer_four():
    keys = [str(x) for x in range(2006, 2016)]

    Top15 = answer_one()
    Top15["Average GDP"] = Top15[keys].mean(axis=1)

    target = Top15.sort_values("Average GDP", ascending=False).iloc[5]

    return target["2016"] - target["2010"]

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [235]:
def answer_five():
    Top15 = answer_one()
    return Top15["Energy Supply per Capita"].mean().item()


### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [236]:
def answer_six():
    ren_key = "% Renewable"
    Top15 = answer_one()
    top_ren = Top15.sort_values([ren_key], ascending=False).iloc[0]

    return (top_ren.name, top_ren.loc[ren_key])


### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [270]:
def answer_seven():
    Top15 = answer_one()
    Top15['Citation Ratio'] =  Top15['Self-citations'] / Top15['Citations']
    top_ratio = Top15.sort_values(['Citation Ratio'], ascending=False).iloc[0]

    return (top_ratio.name, top_ratio['Citation Ratio'])


('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [245]:
def pop_estimate(Top15):
    Top15["Population Estimate"] = Top15["Energy Supply"] / Top15["Energy Supply per Capita"]

    return Top15

def answer_eight():
    Top15 = pop_estimate(answer_one())
    sorted_by_pop_estimate = Top15.sort_values(["Population Estimate"], ascending=False)
    
    return sorted_by_pop_estimate.iloc[2].name


### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [246]:
def answer_nine():
    Top15 = pop_estimate(answer_one())

    Top15["Energy Supply per Capita"] = Top15["Energy Supply per Capita"].astype(np.float)
    Top15["Citable Documents per Person"] = (Top15["Citable documents"] / Top15["Population Estimate"]).astype(np.float)

    correlation = Top15.corr()
    return correlation["Citable Documents per Person"].loc["Energy Supply per Capita"]


### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [247]:
def is_high_renew(row):
    return 1 if row > median else 0

def answer_ten():
    Top15 = answer_one()
    median = Top15["% Renewable"].median()
    Top15["HighRenew"] = Top15["% Renewable"].apply(is_high_renew)
    
    return Top15["HighRenew"]

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [248]:
def add_continents(Top15):
    ContinentDict  = c = {'China':'Asia', 
              'United States':'North America', 
              'Japan':'Asia', 
              'United Kingdom':'Europe', 
              'Russian Federation':'Europe', 
              'Canada':'North America', 
              'Germany':'Europe', 
              'India':'Asia',
              'France':'Europe', 
              'South Korea':'Asia', 
              'Italy':'Europe', 
              'Spain':'Europe', 
              'Iran':'Asia',
              'Australia':'Australia', 
              'Brazil':'South America'}
    Top15["Continent"] = Top15.apply(lambda s: c[s.name] if s.name in c else s, axis=1)
    return Top15

def answer_eleven():
    Top15 = add_continents(pop_estimate(answer_one()))
    Top15["Population Estimate"] = Top15["Population Estimate"].astype(np.float)
    
    return Top15.groupby(["Continent"]).agg(["sum", "size", "mean", "std"])["Population Estimate"]

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [249]:
def answer_twelve():
    Top15 = add_continents(answer_one())

    Top15["% Renewable"] = pd.cut(Top15["% Renewable"], 5)
    return Top15.groupby(["Continent", "% Renewable"]).agg("size")

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [250]:
def answer_thirteen():
    Top15 = pop_estimate(answer_one())
    formatted = Top15["Population Estimate"].apply(lambda s: "{:,}".format(s))
    # Return a series with the right name just to be safe
    return pd.Series(formatted, name="PopEst")

### Optional

Use the built in function `plot_optional()` to see an example visualization.